<a href="https://colab.research.google.com/github/Varad2001/Gender-classification-simple-project/blob/main/model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np 
import pandas as pd
import keras 
import tensorflow as tf
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Dense, Dropout 
from keras.layers import Flatten, BatchNormalization
from tensorflow.keras.utils import to_categorical
import os
from PIL import Image
from PIL import UnidentifiedImageError

In [3]:
# load the model from the corresponding directory
!ls /content/drive/MyDrive/Colab_Notebooks/saved_model/my_model1
new_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab_Notebooks/saved_model/my_model1')

assets	keras_metadata.pb  saved_model.pb  variables


In [4]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 50, 50, 1)        4         
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 50, 50, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 25, 25, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 25, 25, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 25, 25, 32)       128       
 hNormalization)                                                 
                                                        

Testing:<br>
1. Extract all images, convert them to nd-array and then add them to another nd-array.<br>
2. Reshape the resulting nd-array and get predictions from the model.<br>
3. Extract the probabilities for male or female label. <br>
4. e.g. If predicting male images, extract the first elements of the predictions, count how many are greater than 0.6-males, how many are less than 0.4- females and remaining - confused. <br>
5. Calculate the accuracy.

for every image, a list containing two elements is returned: <br>
first element : prob that the image belongs to male category<br>
second element : prob that the image belongs to female category<br>
e.g. <br>
[0.8, 0.2] means that the model thinks that there is 80% chance that the given image is male. <br>

Thus, we will extract the values of first elements for each image and compute the result as follows:<br>
if the value > 0.6 ==> the prediction is 'male'<br>
if the value < 0.4 ==> the prediction is 'female' <br>
else the model is confused

In [7]:
# define the function which converts imgs into arrays and performs the necessary preprocessing
# input : Image object , returns : 3-dim nd-array with shape(img_width, img_height, 1)

def convert_img(img1):
    img = img1.convert("L")         # convert the image into greyscale
    px = 50
    img = img.resize((px,px))       # resize the image to 50*50 pixels, ==> total of 2500 pixels
    img = np.asarray(img)           # convert the image into numpy array, ==> shape will be (50,50)
    img = img / 255.0               # scale the pixel values between 0 to 1
    img = img.reshape(px,px,1)      # convert the two dimensional array to three-dimensional array
    return img

In [46]:
, UnidentifiedImageError
# function to predict the category and calculate accuracy
# input 'img' is given as array of shape (width, height, 3)
# returns dictionary containing probabilities

def predict_gender(img):
    imgs = np.array([])
    img = Image.fromarray(img)          # create image from input 'img' array
    img = convert_img(img)              # perform preprocessing
    imgs = np.append(imgs,img)          # store the array 
    imgs = imgs.reshape(1,50,50,1)      # input to the model must be provided as an array of shape : (no_of_imgs, px, px, 1)

    prediction = new_model.predict(imgs)
    male_prob = float(prediction[0][0])        # extract first element of prediction array which represents the prob of image being 'male' 
    female_prob = float(prediction[0][1])      # extract female prob
    results = {'Male' : male_prob, 'Female' : female_prob}
    if male_prob > 0.5:
        print("Image category:Male")
    else :
        print("Image category:Female")
    return results
  
  

In [42]:
# specify the path of the image
img_file_path = '/content/Chris-Evans-buzz-cut-e1538974047133.jpg'

In [44]:
img = Image.open(img_file_path)
img = np.asarray(img)
img.shape

(2171, 1401, 3)

In [47]:
results  = predict_gender(img)
results

Image category:Male


{'Female': 0.06721476465463638, 'Male': 0.932785153388977}